# Data process

Suppose submodule is pulled, we now

1. Download label data from the website.
2. Parse those data, filter out useless properties, and save them as a csv file.
   - each data item contains 
     1. the btor2 file name
     2. the validator
     3. the result

In [1]:
import urllib
# user should set their own proxy...
proxies = {'https': 'http://172.31.80.1:7890', 'http': 'http://172.31.80.1:7890', 'all': 'sock5://172.31.80.1:7891'}

**Download and decompress the labels.**

In [3]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.cip.ifi.lmu.de/~chien/benchexec-results/btor2c-eval/final-full-results/"

# Send a GET request to the URL
response = requests.get(url, proxies=proxies)

print(response)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all <a> tags that contain the file links
file_links = soup.find_all("a")

# Extract the href attribute from each <a> tag
file_names = [link.get("href") for link in file_links]

<Response [200]>


In [4]:
tools = ["cpachecker", "esbmc", "veriabs"]

# Define the regular expression pattern
# <tool_name>-lazyMod.*.results.*.bv-64.xml.bz2
pattern = fr"({'|'.join(tools)})-lazyMod.*.results.*.bv-64.xml.bz2"
print(pattern)
# Filter file_names using the pattern
filtered_file_names = [file_name for file_name in file_names if re.fullmatch(pattern, file_name)]

# Print the filtered file names
for file_name in filtered_file_names:
    print(file_name)

(cpachecker|esbmc|veriabs)-lazyMod.*.results.*.bv-64.xml.bz2
cpachecker-lazyMod.2023-01-17_17-25-41.results.predAbs.bv-64.xml.bz2
cpachecker-lazyMod.2023-01-18_01-24-33.results.bmc-boolector-noPtrAlias.bv-64.xml.bz2
esbmc-lazyMod.2023-01-17_19-41-45.results.kind.bv-64.xml.bz2
esbmc-lazyMod.2023-01-18_02-08-43.results.bmc.bv-64.xml.bz2
veriabs-lazyMod.2023-01-17_22-40-24.results.default.bv-64.xml.bz2


In [9]:
import os
import urllib.request

base_url = "https://www.cip.ifi.lmu.de/~chien/benchexec-results/btor2c-eval/final-full-results/"
label_directory = "labels/"

for file_name in filtered_file_names:
    url = base_url + file_name
    destination = label_directory + file_name
    
    if not os.path.exists(destination):
        urllib.request.urlretrieve(url, destination)

In [13]:
import bz2
import os

# Loop through each bz2 file in the directory
for file_name in filtered_file_names:
    bz2_file_path = label_directory + file_name
    extract_path = label_directory + file_name[:-4]  # Remove the .bz2 extension
    
    # Open the bz2 file
    with bz2.open(bz2_file_path, 'rb') as bz2_file:
        # Read the compressed data
        data = bz2_file.read()
        
        # Write the decompressed data to a new file
        with open(extract_path, 'wb') as extract_file:
            extract_file.write(data)


**parse the xml label files to csv files.**

In [59]:
import yaml

# Load the config.yaml file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the value of dataset.path
dataset_path = config['dataset']['directory']

# Print the dataset path
print(dataset_path)

word-level-hwmc-benchmarks/bv/btor2/


In [90]:
xml_label_files = [label_directory + file_name[:-4] for file_name in filtered_file_names]

import xml.etree.ElementTree as ET

runs = []

for xml_file in xml_label_files:
    run = []
    tree = ET.parse(xml_file)
    root = tree.getroot()
    run_children = root.findall('run')
    for run_node in run_children:
        parent_dir = os.path.basename(os.path.dirname(run_node.attrib['files'])) + "/"
        item = {
            'yml': dataset_path + parent_dir + os.path.basename(run_node.attrib['name']),
            'btor2': dataset_path + parent_dir + os.path.basename(run_node.attrib['files'][:-2]) + 'btor2',
            'c': dataset_path + parent_dir + os.path.basename(run_node.attrib['files'][:-1]),
            'property': run_node.attrib['properties'],
            'propertyFile': dataset_path + "../../properties/" + os.path.basename(run_node.attrib['propertyFile']),
            'expectedVerdict': run_node.attrib['expectedVerdict'],
        }
        # run_node.attrib
        properties = run_node.findall('column')
        for property_node in properties:
            if property_node.attrib['title'] == 'cputime':
                item['cputime'] = float(property_node.attrib['value'][:-1])
            elif property_node.attrib['title'] == 'memory':
                item['memory'] = int(property_node.attrib['value'][:-1])
            elif property_node.attrib['title'] == 'status':
                item['status'] = \
                    'TRUE' if 'true' in property_node.attrib['value'] else \
                    'FALSE' if 'false' in property_node.attrib['value'] else \
                    'FAILED'
        run.append(item)
    runs.append(run)
# we have 5 files, each of them contains 1342 items
print(runs[0][0])

{'yml': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.yml', 'btor2': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.btor2', 'c': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.c', 'property': 'unreach-call', 'propertyFile': 'word-level-hwmc-benchmarks/bv/btor2/../../properties/unreach-call.prp', 'expectedVerdict': 'false', 'cputime': 901.71803173, 'memory': 346869760, 'status': 'FAILED'}


**Note that we need to invoke btor2c**

In [94]:
for run in runs:
    for test in run:
        if not os.path.exists(test['btor2']):
            print(test['btor2'])
        if not os.path.exists(test['yml']):
            print(test['yml'])
        # if not os.path.exists(test['c']):
        #     print(test['c'])
        if not os.path.exists(test['propertyFile']):
            print(test['propertyFile'])

TODO: cputime, status, memory need to be summarized into one property 

In [100]:
import json
import pickle

# Dump the runs dictionary into a pickle file
with open('runs.pickle', 'wb') as file:
    pickle.dump(runs, file)

{'yml': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.yml', 'btor2': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.btor2', 'c': 'word-level-hwmc-benchmarks/bv/btor2/beem/adding.1.prop1-back-serstep.c', 'property': 'unreach-call', 'propertyFile': 'word-level-hwmc-benchmarks/bv/btor2/../../properties/unreach-call.prp', 'expectedVerdict': 'false', 'cputime': 901.71803173, 'memory': 346869760, 'status': 'FAILED'}
